In [14]:
%pip install amazon-textract-caller amazon-textract-textractor langchain-community pandas

  Using cached amazon_textract_caller-0.2.4-py2.py3-none-any.whl.metadata (7.2 kB)
  Using cached amazon_textract_textractor-1.8.2-py3-none-any.whl.metadata (9.7 kB)
  Using cached amazon_textract_response_parser-1.0.3-py2.py3-none-any.whl.metadata (11 kB)
  Using cached XlsxWriter-3.2.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached editdistance-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
Using cached amazon_textract_caller-0.2.4-py2.py3-none-any.whl (13 kB)
Using cached amazon_textract_textractor-1.8.2-py3-none-any.whl (307 kB)
Using cached amazon_textract_response_parser-1.0.3-py2.py3-none-any.whl (30 kB)
Using cached editdistance-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (401 kB)
Using cached XlsxWriter-3.2.0-py3-none-any.whl (159 kB)
Note: you may need to restart the kernel to use updated packages.


In [55]:
import os

os.environ['AWS_REGION'] = 'us-east-1'
os.environ['S3_BUCKET_URL'] = 's3://financial-statement-extraction/'

fin_document = "Northmarq_Balance_Sheet.pdf"
fin_csv_file = "Northmarq_Balance_Sheet.csv"

In [18]:
import textractcaller as tc
from textractor.parsers import response_parser
import boto3

QUERY_1 = "Choose from the three types to answer the question: balance sheet, income, cash flow. What type is the document?"
QUERY_2 = "What is the company name?"
QUERY_3 = "What is the month or date or year ended?"

def extract_document(filename):
    textract = boto3.client('textract', region_name=os.getenv('AWS_REGION'))
    q = tc.Query(text=QUERY_1, pages=["*"])
    q2 = tc.Query(text=QUERY_2, pages=["*"])
    q3 = tc.Query(text=QUERY_3, pages=["*"])
    
    result = tc.call_textract(
        input_document=f"{os.getenv('S3_BUCKET_URL')}{filename}",
        queries_config=tc.QueriesConfig(queries=[q,q2,q3]),
        adapters_config=None,
        features=[tc.Textract_Features.QUERIES, tc.Textract_Features.TABLES, tc.Textract_Features.LAYOUT],
        force_async_api=True,
        boto3_textract_client=textract
    )
    
    document = response_parser.parse(result)
    return document

In [19]:
def build_tables_dict(document):
    doc_tables = {}
    
    for page in document.pages:
        # what is the doc type and company name?
        company_name = str([q.result for q in page.queries if q.query == QUERY_2][0])
        if company_name not in doc_tables.keys():
            doc_tables[company_name] = {}
        doc_type = str([q.result for q in page.queries if q.query == QUERY_1][0])
        if doc_type not in doc_tables.keys():
            doc_tables[company_name][doc_type] = []
        # extract tables
        for table in page.tables:
            doc_tables[company_name][doc_type].append(table.to_pandas())

    return doc_tables

In [20]:
document = extract_document(fin_document)

In [21]:
import pandas as pd
from IPython.display import display

tables = build_tables_dict(document)
for company in tables:
        print("\n\n----------------------------------------")
        print(f"COMPANY NAME: {company}\n\n")
        for doctype in tables[company]:
            result = pd.concat(tables[company][doctype])
            result = result.reset_index(drop=True)
            print(f"DOCUMENT TYPE: {doctype}\n")
            display(result)



----------------------------------------
COMPANY NAME: Example Corporation


DOCUMENT TYPE: Balance Sheet



,0,1
0,Current assets,
1,Cash and cash equivalents,"$ 2,200"
2,Short-term investments,"10,000"
3,Accounts receivable - net,"39,500"
4,Other receivables,"1,000"
5,Inventory,"31,000"
6,Supplies,"3,800"
7,Prepaid expensees,"1,500"
8,Total current assets,"89,000"
9,Investments,"36,000"


In [27]:
df = tables['Example Corporation']['Balance Sheet'][0]
display(df)

,0,1
0,Current assets,
1,Cash and cash equivalents,"$ 2,200"
2,Short-term investments,"10,000"
3,Accounts receivable - net,"39,500"
4,Other receivables,"1,000"
5,Inventory,"31,000"
6,Supplies,"3,800"
7,Prepaid expensees,"1,500"
8,Total current assets,"89,000"
9,Investments,"36,000"


In [31]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [33]:
df = df.set_index(df.columns[0])

In [38]:
display(df.index)

Index(['Current assets', 'Cash and cash equivalents', 'Short-term investments',
       'Accounts receivable - net', 'Other receivables', 'Inventory',
       'Supplies', 'Prepaid expensees', 'Total current assets', 'Investments',
       'Property, plant & equipment', 'Land', 'Land improvements', 'Buildings',
       'Equipment', 'Less: accumulated depreciation',
       'Property, plant & equip. - net', 'Intangible assets', 'Goodwill',
       'Other intangible assets', 'Total intangible assets', 'Other assets',
       'Total assets'],
      dtype='object', name=0)

In [39]:
import pandas as pd

def get_fieldnames(csv_file):
    df = pd.read_csv(csv_file, sep=',')
    return df.iloc[:, 0].values

In [56]:
sqlprompt = f"""
Instructions: Use only the field names provided in the <content> tags to perform your task.
<content>
{df.index}
</content>

<schema>
{get_fieldnames('test.txt')}
</schema>

Task: Write SQL queries that retrieve values for each field from the schema provided.
Each SQL query should have a condition WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'.
Then match one SQL query to the field from the content provided.
Provided the response as a tuple output as follows:
("content field", "schema field", "SQL Query")
Provide the output in between <output> tags.
"""

print(sqlprompt)


Instructions: Use only the field names provided in the <content> tags to perform your task.
<content>
Index(['Current assets', 'Cash and cash equivalents', 'Short-term investments',
       'Accounts receivable - net', 'Other receivables', 'Inventory',
       'Supplies', 'Prepaid expensees', 'Total current assets', 'Investments',
       'Property, plant & equipment', 'Land', 'Land improvements', 'Buildings',
       'Equipment', 'Less: accumulated depreciation',
       'Property, plant & equip. - net', 'Intangible assets', 'Goodwill',
       'Other intangible assets', 'Total intangible assets', 'Other assets',
       'Total assets'],
      dtype='object', name=0)
</content>

<schema>
['cash_and_equiv' 'short_term_invest' 'net_account_receiv' 'inventory'
 'supplies' 'total_current_assets']
</schema>

Task: Write SQL queries that retrieve values for each field from the schema provided.
Each SQL query should have a condition WHERE company_name = 'sample company' AND financial_quarter = 'fi

In [57]:
import boto3

client = boto3.client('bedrock-runtime', region_name='us-east-1')
response = client.converse(
    modelId='anthropic.claude-3-sonnet-20240229-v1:0',
    messages=[{'role': 'user', 'content': [{'text': sqlprompt}]}],
    inferenceConfig={'temperature': 1}
)

output = response['output']['message']['content'][0]['text']

print(output)

<output>
("Cash and cash equivalents", "cash_and_equiv", "SELECT cash_and_equiv FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'")
("Short-term investments", "short_term_invest", "SELECT short_term_invest FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'")
("Accounts receivable - net", "net_account_receiv", "SELECT net_account_receiv FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'")
("Inventory", "inventory", "SELECT inventory FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'")
("Supplies", "supplies", "SELECT supplies FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'")
("Total current assets", "total_current_assets", "SELECT total_current_assets FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'")
</output>


In [58]:
import re

def extract_output_text(input_text):
    # Use regular expression to find text between <output> and </output> tags
    pattern = r"<output>(.*?)<\/output>"
    match = re.search(pattern, input_text, re.DOTALL)
    
    if match:
        return match.group(1).strip()
    else:
        return None

def parse_tuples(input_string):
    input_string = input_string.replace('"', '')
    # Remove leading/trailing whitespace and split by newline
    lines = input_string.strip().split('\n')
    # Parse each line as a tuple
    result = [tuple(line.strip('(),').split(', ')) for line in lines]
    return result

In [59]:
text = extract_output_text(output)
sql_tuples_list = parse_tuples(text)
print(sql_tuples_list)

[('Cash and cash equivalents', 'cash_and_equiv', "SELECT cash_and_equiv FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'"), ('Short-term investments', 'short_term_invest', "SELECT short_term_invest FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'"), ('Accounts receivable - net', 'net_account_receiv', "SELECT net_account_receiv FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'"), ('Inventory', 'inventory', "SELECT inventory FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'"), ('Supplies', 'supplies', "SELECT supplies FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'"), ('Total current assets', 'total_current_assets', "SELECT total_current_assets FROM table WHERE company_name = 'sample company' AND financial_quarter = 'first_quarter'")]


In [60]:
def database_retrieval(tuples_list, extracted_data):
    database = pd.read_csv("test.txt", sep=',', index_col=0)
    for item in tuples_list:
        extracted_vals = extracted_data.loc[item[0]].values
        db_vals = database.loc[item[1]].values
        for i in range(len(db_vals)):
            print(f"Database value: {db_vals[i]}, Extracted value: {extracted_vals[i]}")
        print("----------------")

In [61]:
database_retrieval(sql_tuples_list, df)

Database value: $ 2,200 , Extracted value: $ 2,200
----------------
Database value: 10,000 , Extracted value: 10,000
----------------
Database value: 39,500 , Extracted value: 39,500
----------------
Database value: 31,000 , Extracted value: 31,000
----------------
Database value: 3,800 , Extracted value: 3,800
----------------
Database value: 89,000 , Extracted value: 89,000
----------------
